In [3]:
from ortools.sat.python import cp_model
import numpy as np

In [4]:
# Define Input Data
intervals = [(1,10), (1,15), (11,20), (16,20), (15,21)]
num_resources = 2
# intervals_input = [(1,3), (4,11)]

In [49]:
''' One Resource Solution '''

# Declare Model
model = cp_model.CpModel()

# Define Variables

# resource assigned to each task/interval
assign_resource = [model.NewIntVar(0, num_resources, f'task_assign{i+1}') for i in range(len(intervals))]

# variables for intervals if present or not
scheduled = [ model.NewOptionalIntervalVar(i[0], i[1]-i[0], i[1], assign_resource[e], f'interval{e+1}') 
              for e, i in enumerate(intervals) ]

# No Overlapping Intervals
model.AddNoOverlap(scheduled)

# Maximize Length of Schedule
model.Maximize(sum(assign_resource))

In [88]:
''' Multiple Resource Numpy Solution '''

# Declare Model
model = cp_model.CpModel()

# Define Variables
num_intervals = len(intervals)

# 2D boolean table for interval-resource assignments
# row represents resource | column represents interval
assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
for i in range(num_intervals):
  for r in range(num_resources):
    assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

# 2D array for schedules | row represents a resource
schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
for i in range(num_intervals):
  interval = intervals[i]
  for r in range(num_resources):
    schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                      interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

# No overlapping intervals
for r in range(num_resources):
  model.AddNoOverlap(schedules[r])

# Each task scheduled at most once
for i in range(num_intervals): 
  model.AddAtMostOne(assign_resource[:,i])

# Maximize Length of Schedule
model.Maximize(sum(assign_resource.flatten()))

In [18]:
''' Multiple-Objective Solution '''

# Declare Model
model_1 = cp_model.CpModel()
model_2 = cp_model.CpModel()

# Define Variables
num_intervals = len(intervals)

# 2D boolean table for interval-resource assignments
# row represents resource | column represents interval
assign_resource_1 = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
assign_resource_2 = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
durations = np.zeros((num_resources, num_intervals))
for i in range(num_intervals):
  for r in range(num_resources):
    assign_resource_1[r][i] = model_1.NewBoolVar(f'assign_i{i+1}_r{r+1}')
    assign_resource_2[r][i] = model_2.NewBoolVar(f'assign_i{i+1}_r{r+1}')
    durations[r][i] 

# 2D array for schedules | row represents a resource
schedules_1 = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
schedules_2 = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
for i in range(num_intervals):
  interval = intervals[i]
  for r in range(num_resources):
    durations[r][i] = interval[1]-interval[0]
    schedules_1[r][i] = model_1.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource_1[r][i], f'interval_i{i+1}_r{r+1}')
    schedules_2[r][i] = model_2.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource_2[r][i], f'interval_i{i+1}_r{r+1}')

# No overlapping intervals
for r in range(num_resources):
  model_1.AddNoOverlap(schedules_1[r])
  model_2.AddNoOverlap(schedules_2[r])

# Each task scheduled at most once
for i in range(num_intervals): 
  model_1.AddAtMostOne(assign_resource_1[:,i])
  model_2.AddAtMostOne(assign_resource_2[:,i])

# Maximize Length of Schedule
model_1.Maximize(sum(assign_resource_1.flatten()))

# Invoke Intermediate Solver
solver_1 = cp_model.CpSolver()
status_1 = solver_1.Solve(model_1)
opt_val = int(solver_1.ObjectiveValue()) 

# Add Optimal Value for Solver 2
model_2.Add(sum(assign_resource_1.flatten()) == opt_val)
model_2.Maximize(cp_model.LinearExpr.WeightedSum(assign_resource_2.flatten(), durations.flatten()))

In [19]:
# Invoke Solver
solver = cp_model.CpSolver()
status = solver.Solve(model_2)

In [21]:
# Display Results

# STATUS CODES = {OPTIMAL: 4, FEASIBLE: 2, INFEASIBLE: 3, MODEL_INVALID: 1, UNKNOWN: 0}
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
  print(f'Intervals Scheduled: {solver.ObjectiveValue()}')
  print('--------')
  # print schedules by resource
  for r in range(num_resources):
    scheduled_intervals = []
    for i in range(num_intervals):
      is_scheduled = solver.Value(assign_resource_2[r][i])
      if is_scheduled: scheduled_intervals.append(intervals[i])
    print(f'Resource {r+1}:', scheduled_intervals)


Intervals Scheduled: 38.0
--------
Resource 1: [(1, 10), (11, 20)]
Resource 2: [(1, 15), (15, 21)]
